<a href="https://colab.research.google.com/github/SavvinaDaniil/BiasInRecommendation/blob/main/2.%20Book%20Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

This notebook should be run on Google Colab.

# Process
In this notebook, I will train the book recommendation algorithms using two different packages: <a href="http://surpriselib.com/">Surprise</a> & <a href="https://cornac.readthedocs.io/en/latest/">Cornac</a>. 

## A. Import libraries

In [1]:
!pip install cornac
!pip install surprise

     |████████████████████████████████| 12.4 MB 4.5 MB/s 
     |████████████████████████████████| 11.8 MB 6.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630167 sha256=987c8ebc8475b4c94f578432e2708fb0a4931c322c2f32d98b31c578ced3c9c9
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [52]:
%tensorflow_version 1.x
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import matplotlib.pyplot as plt
import random as rd
import pandas as pd
import numpy as np
#from run_algorithms import train_algorithms, train_algorithms_kf, prepare_dataset, prepare_dataset_kf
from tqdm import tqdm
import time
pd.set_option("display.precision", 6)

# Cornac imports
import cornac
from cornac.eval_methods import RatioSplit
from cornac.data import Reader as CornacReader #Reader exists in both packages
from cornac.models import MostPop, MF, PMF, BPR, NeuMF, WMF, HPF, VAECF, NMF
from cornac.models import NMF as CornacNMF #NMF exists in both packages
from cornac.metrics import MAE, MSE, RMSE, Precision, Recall, NDCG, AUC, MAP, FMeasure, MRR

from surprise import BaselineOnly, KNNBasic, KNNWithMeans, SVDpp, SVD
from surprise import NMF as SurpriseNMF #NMF exists in both packages
from surprise import Dataset
from surprise import Reader as SurpriseReader #Reader exists in both packages
from surprise.model_selection import train_test_split, KFold, GridSearchCV
from surprise import accuracy
from sklearn.preprocessing import MinMaxScaler

from collections import defaultdict
from scipy import stats
from numpy.linalg import norm
import seaborn as sns
# set plot style: grey grid in the background:
sns.set(style="darkgrid")
pd.set_option("display.precision", 8)

## B. Set hyperparameters
There are certain hyperparameters that need to be tuned before the run. 

In [3]:
item_threshold = 5 # remove users with less than item_threshold items
user_threshold = 5 # remove items with less than user_threshold users
top_threshold = 200 # remove users who have rated more than top_threshold items
recommendation_type = "books" # books, music or movies
item_col = "book" # the item column
my_seed = 0 # random_seed
top_fraction_items = 0.2 # the limit for an item to be considered popular
top_fraction_users = 0.2# the limit for a user to be considered High Mainstriminess
split_by = "pop_fraq" # sort users by fraction of popular items (pop_fraq) or by average popularity in profile (pop_item_fraq)
test_size = 0.2 # the percentage of "hold out" data that are used for testing
rating_threshold = 1.0 # needed for the cornac library
predict_col = "rating" # the column we are predicting
train_way = "simple_split"
n_splits = 5 # the amount of splits

if train_way == "simple_split": n_splits = 1
rd.seed(my_seed)
np.random.seed(my_seed)

These additions will be useful so we can load and save the different files (plots and processed data) with clarity on the hyperparameters.

In [4]:
addition_1 = "_u"+str(item_threshold)+"_i"+str(user_threshold)+"_t"+str(top_threshold)
addition_2 = addition_1 + "_tfi"+str(int(100*top_fraction_items))
addition_3 = addition_2 + "_tfu"+str(int(100*top_fraction_users))
addition_4 = addition_3 + ("_sbpf" if (split_by=="pop_fraq") else "_sbpif")

## C. Read files

In [5]:
user_events_file = "https://raw.githubusercontent.com/SavvinaDaniil/BiasInRecommendation/main/data/processed/"+recommendation_type+"/ratings"+addition_1+".csv"
high_user_file = "https://raw.githubusercontent.com/SavvinaDaniil/BiasInRecommendation/main/data/processed/"+recommendation_type+"/high_users"+addition_4+".csv"
low_user_file = "https://raw.githubusercontent.com/SavvinaDaniil/BiasInRecommendation/main/data/processed/"+recommendation_type+"/low_users"+addition_4+".csv"
medium_user_file = "https://raw.githubusercontent.com/SavvinaDaniil/BiasInRecommendation/main/data/processed/"+recommendation_type+"/med_users"+addition_4+".csv"
df_item_dist_file = "https://raw.githubusercontent.com/SavvinaDaniil/BiasInRecommendation/main/data/processed/"+recommendation_type+"/item_pop_dist"+addition_1+".csv"

In [6]:
male_oriented_file = "https://raw.githubusercontent.com/SavvinaDaniil/BiasInRecommendation/main/data/processed/books/high_users_u5_i5_t200_tfu20_mfd.csv"
diverse_file = "https://raw.githubusercontent.com/SavvinaDaniil/BiasInRecommendation/main/data/processed/books/med_users_u5_i5_t200_tfu20_mfd.csv"
female_oriented_file = "https://raw.githubusercontent.com/SavvinaDaniil/BiasInRecommendation/main/data/processed/books/low_users_u5_i5_t200_tfu20_mfd.csv"

In [7]:
low = pd.read_csv(low_user_file, index_col=0)
med = pd.read_csv(medium_user_file, index_col=0)
high = pd.read_csv(high_user_file, index_col=0)

In [8]:
male_oriented = pd.read_csv(male_oriented_file, index_col=0)
diverse = pd.read_csv(diverse_file, index_col=0)
female_oriented = pd.read_csv(female_oriented_file, index_col=0)

In [9]:
num_users1 = len(low) + len(med) + len(high)
num_users2 = len(male_oriented) + len(diverse) + len(female_oriented)
print(num_users1, num_users2)
num_users = num_users1

6358 6358


In [10]:
df_item_dist = pd.read_csv(df_item_dist_file, index_col = 0)

In [11]:
df_item_dist.index

Int64Index([ 124,   70,   50,  465,  581,  192,   20,  657,  434,  127,
            ...
            4510, 4508, 4505, 4500, 4499, 4498, 4497, 4494, 4490, 6920],
           dtype='int64', length=6921)

## D. Recommendation 

In [12]:
# we need two df_item_dist: one for Surprise, one for Cornac
df_item_dist_Surprise = df_item_dist.copy()

### Surprise

In [ ]:
# df_events = pd.read_csv(user_events_file, low_memory = False, header=0) # create dataframe

In [ ]:
# # load dataset in Surprise
# reader = SurpriseReader(rating_scale=(df_events[predict_col].min(), df_events[predict_col].max()))
# data = Dataset.load_from_df(df_events, reader)

In [ ]:
# # split in train and test set
# trainset, testset = train_test_split(data, test_size = test_size, random_state = 0)

In [ ]:
# # select and initialize algorithms
# algo_names = ["Random",
#               "MostPopular",
#               #'UserItemAvg',
#                   'UserKNN',
#                   'UserKNNAvg',
#                   'NMF', 
#                   'SVD']

# # the default parameters for all algorithms
# algos = [] 
# algos.append(None)#Random())
# algos.append(None)#MostPopular())
# #algos.append(BaselineOnly()) 
# algos.append(KNNBasic(sim_options = {'name': 'cosine', 'user_based': True})) 
# algos.append(KNNWithMeans(sim_options = {'name': 'cosine', 'user_based': True})) 
# algos.append(SurpriseNMF())
# algos.append(SVD())

In [ ]:
# def get_top_n_Surprise(predictions, n=10):
#     # First map the predictions to each user.
#     top_n = defaultdict(list)
#     for uid, iid, true_r, est, _ in predictions:
#         top_n[uid].append((iid, est))
#     # Then sort the predictions for each user and retrieve the k highest ones.
#     for uid, user_ratings in top_n.items():
#         user_ratings.sort(key=lambda x: x[1], reverse=True)
#         top_n[uid] = user_ratings[:n]
#     return top_n

In [ ]:
# def get_top_n_random_Surprise(testset, df_item_dist, n=10):
#     top_n = defaultdict(list)
#     for uid, iid, true_r in testset:
#       if len(top_n[uid]) == 0:
#         for i in range(0, 10):
#           top_n[uid].append((rd.choice(df_item_dist.index), i))
    
#     return top_n

In [ ]:
# def get_top_n_mp_Surprise(testset, item_dist, n=10):
#     top_n = defaultdict(list)
#     for uid, iid, true_r in testset:
#         if len(top_n[uid]) == 0:
#             for iid, count in df_item_dist[:n]["count"].items():
#                 top_n[uid].append((iid, count))
#     return top_n

In [ ]:
# trainset_for_testing = trainset.build_anti_testset() + trainset.build_testset()

In [ ]:
# len(trainset_for_testing)

In [ ]:
# trainset2 = trainset.build_anti_testset()

In [ ]:
# i = 0
# low_rec_gap_list = [] # one entry per algorithm
# medium_rec_gap_list = []
# high_rec_gap_list = []
# start = time.time()

# for i in tqdm(range(0, len(algo_names))): # for every algorithm
#     print("~~~~~~~~~~~~~~~~NEW~~~~~~~~~~~~~~~~~")
#     df_item_dist_Surprise[algo_names[i]] = 0 # I am adding a column to Surprise
#     low_rec_gap = 0
#     medium_rec_gap = 0
#     high_rec_gap = 0
    
#     # get accuracy for personalized approaches
#     if algo_names[i] != 'Random' and algo_names[i] != 'MostPopular': # for proper algorithms
#         algos[i].fit(trainset) # fit
#         predictions = algos[i].test(trainset2) # predict
#         print(algo_names[i]) # end of fitting

#         #get_mae_of_groups(predictions, low, med, high) TO BE ADDED
    
#     # get top-n items and calculate gaps for all algorithms
#     if algo_names[i] == 'Random':
#         top_n = get_top_n_random_Surprise(trainset2, df_item_dist, n=10)
#         print(algo_names[i])
#     elif algo_names[i] == 'MostPopular':
#         top_n = get_top_n_mp_Surprise(trainset2, df_item_dist, n=10)
#         print(algo_names[i])
#     else:
#         top_n = get_top_n_Surprise(predictions, n=10)

#     # calculate GAPs
#     low_count = 0
#     med_count = 0
#     high_count = 0
#     for uid, user_ratings in top_n.items():
#         iid_list = []
#         for (iid, _) in user_ratings:
#             df_item_dist_Surprise.loc[iid, algo_names[i]] += 1
#             iid_list.append(iid)
#         gap = sum(df_item_dist_Surprise["count"].loc[iid_list]) / len(iid_list)
#         if uid in low.index:
#             low_rec_gap += gap
#             low_count += 1
#         elif uid in med.index:
#             medium_rec_gap += gap
#             med_count += 1
#         elif uid in high.index:
#             high_rec_gap += gap
#             high_count += 1
#     low_rec_gap_list.append(low_rec_gap / low_count)
#     medium_rec_gap_list.append(medium_rec_gap / med_count)
#     high_rec_gap_list.append(high_rec_gap / high_count)
#     i += 1 # next algorithm
#     end = time.time()
#     print("It took " + str(np.round(end-start)) + " seconds.")
#     start = time.time()

In [ ]:
# Not implemented here yet.
# if train_way == "kfold":
#     for alg in algo_names:
#         df_item_dist[alg] = df_item_dist[alg]/n_splits

In [ ]:
# #len(np.unique([x[0] for x in trainset_for_testing])),
# len(np.unique([x[0] for x in testset]))  # unique users


In [ ]:
# #len(np.unique([x[1] for x in trainset_for_testing])),
# len(np.unique([x[1] for x in testset])) # unique items

## Save files
To save:
1. df_item_dist
2. low_rec_gap_list etc

In [ ]:
# save = True

In [ ]:
# if save:
#   from google.colab import drive
#   import pickle as pkl
#   drive.mount('/content/drive')

#   df_item_dist_Surprise.to_csv("/content/drive/My Drive/item_pop_dist"+addition_1+"_results_Surprise.csv")
#   with open("/content/drive/My Drive/low_rec_gap_list_Surprise"+addition_4+".pkl","wb") as f:
#     pkl.dump(low_rec_gap_list,f)
#   with open("/content/drive/My Drive/med_rec_gap_list_Surprise"+addition_4+".pkl","wb") as f:
#     pkl.dump(medium_rec_gap_list,f)
#   with open("/content/drive/My Drive/high_rec_gap_list_Surprise"+addition_4+".pkl","wb") as f:
#     pkl.dump(high_rec_gap_list,f)

### Cornac

In [13]:
# load dataset in Cornac
os.system("wget "+user_events_file)
reader = CornacReader()
data = reader.read(user_events_file.split("/")[-1],sep =",", skip_lines =1)

In [14]:
# Split the data based on ratio
rs = RatioSplit(data=data, test_size=test_size, rating_threshold=rating_threshold, seed=0)

In [15]:
len([x for x in rs.__dict__["train_set"].item_data])

6921

In [16]:
# initialize models, here we are comparing: simple, traditional, and neural networks based models
models = [
          # 1: Random
          # 2: MostPop
          MostPop(),
          # 3: UserKNN
          # 4: BPR
          BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123),
          # 5: MF
          MF(k=30, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=123),
          # 6: PMF
          PMF(k=10, max_iter=100, learning_rate=0.001, lambda_reg=0.001),
          # 7: NMF
          NMF(k=15, max_iter=50, learning_rate=0.005, lambda_u=0.06, lambda_v=0.06, lambda_bu=0.02, lambda_bi=0.02, use_bias=False, verbose=True, seed=123),
          # 8: WMF
          WMF(k=50, max_iter=50, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, verbose=True, seed=123),
          # 9: PF
          HPF(k=50, seed=123, hierarchical=False, name="PF"),
          # 10: NueMF
          NeuMF(num_factors=8, layers=[32, 16, 8], act_fn="tanh", num_epochs=1, num_neg=3, batch_size=256, lr=0.001, seed=42, verbose=True),
          # 11: VAECF
          VAECF(k=10, autoencoder_structure=[20], act_fn="tanh", likelihood="mult", n_epochs=100, batch_size=100, learning_rate=0.001, beta=1.0, seed=123, use_gpu=True, verbose=True)
          ]

In [17]:
# define metrics to evaluate the models
metrics = [MAE(), MSE(), RMSE(), AUC(), MAP(), MRR(), 
           Precision(k=5), Precision(k=10), Precision(k=20), Precision(k=50),
           Recall(k=5), Recall(k=10), Recall(k=20), Recall(k=50),
           NDCG(k=5), NDCG(k=10), NDCG(k=20), NDCG(k=50),
           FMeasure(k=5), FMeasure(k=10), FMeasure(k=20), FMeasure(k=50)]

# put it together in an experiment, voilà!
exp = cornac.Experiment(eval_method=rs, models=models, metrics=metrics, user_based=True)
exp.run()

  0%|          | 0/50 [00:00<?, ?it/s]

Optimization finished!


  0%|          | 0/50 [00:00<?, ?it/s]

Learning completed!
Learning...
Learning completed!


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


TEST:
...
        |    MAE |     MSE |   RMSE |    AUC |  F1@10 |  F1@20 |   F1@5 |  F1@50 |    MAP |    MRR | NDCG@10 | NDCG@20 | NDCG@5 | NDCG@50 | Precision@10 | Precision@20 | Precision@5 | Precision@50 | Recall@10 | Recall@20 | Recall@5 | Recall@50 | Train (s) | Test (s)
------- + ------ + ------- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------ + ------- + ------------ + ------------ + ----------- + ------------ + --------- + --------- + -------- + --------- + --------- + --------
MostPop | 2.5202 | 10.0391 | 2.7988 | 0.6544 | 0.0105 | 0.0100 | 0.0099 | 0.0091 | 0.0147 | 0.0350 |  0.0175 |  0.0223 | 0.0141 |  0.0334 |       0.0076 |       0.0063 |      0.0092 |       0.0051 |    0.0253 |    0.0397 |   0.0151 |    0.0820 |    0.0201 |  32.7793
BPR     | 6.5658 | 46.4381 | 6.6616 | 0.6775 | 0.0105 | 0.0102 | 0.0099 | 0.0092 | 0.0148 | 0.0350 |  0.0175 |  0.0226 | 0.0141 |  0.0334 |       0.0076 |       0.0064 |      0.0092 |     

In [18]:
def compute_user_knn(C):
  ctime = time.time()
  print("Training User-based Collaborative Filtering...", )

  sim = C.dot(C.T)
  norms = [norm(C[i]) for i in range(C.shape[0])]

  for i in tqdm(range(C.shape[0])):
    sim[i][i] = 0.0
    for j in range(i+1, C.shape[0]):
      sim[i][j] /= (norms[i] * norms[j])
      sim[j][i] /= (norms[i] * norms[j])

  print("Done. Elapsed time:", time.time() - ctime, "s")
  rec_score = sim.dot(C)
  return rec_score

In [19]:
def read_training_data():
  training_matrix = np.zeros((rs.train_set.matrix.shape[0], rs.train_set.matrix.shape[1]))
  for uid in tqdm(rs.train_set.uid_map.values()):
    for iid in rs.train_set.iid_map.values():
      training_matrix[uid, iid] = rs.train_set.matrix[uid, iid]
  return training_matrix

In [20]:
# creating users-books rating matrix (will be used for User-KNN algorithm)
training_matrix = read_training_data()

100%|██████████| 6358/6358 [29:18<00:00,  3.62it/s]


In [21]:
len(training_matrix), len(training_matrix[0])

(6358, 6921)

In [22]:
# running User-KNN algorithms and getting the user-book scores
user_knn_scores = compute_user_knn(training_matrix)

Training User-based Collaborative Filtering...


100%|██████████| 6358/6358 [00:23<00:00, 273.14it/s] 


Done. Elapsed time: 31.670928239822388 s


In [23]:
# UserKNN recommendation algorithm
def get_top_n_UserKNN(n=10):
    print("User-KNN model is selected:")
    top_n = defaultdict(list)
    # test_items = list(rs.test_set.iid_map.keys())
    for uid in rs.train_set.uid_map.values():
      user_id = list(rs.train_set.user_ids)[uid]
      top_n_items_idxs = list(reversed(user_knn_scores[uid].argsort()))[:n]
      for iid in top_n_items_idxs:
        item_id = list(rs.train_set.item_ids)[iid]
        top_n[int(user_id)].append((int(item_id), user_knn_scores[uid][iid]))
    return top_n

In [24]:
# for model in exp.models:
#   print(model.name, len(model.rank(0)[0]))

In [25]:
def get_top_n(algo_name, n=10):
  for model in exp.models:
    if model.name == algo_name:
      print(model.name + " model is selected:")
      top_n = defaultdict(list)
      for uid in model.train_set.uid_map.values():
        user_id = list(model.train_set.user_ids)[uid]
        try:
          item_rank = model.rank(user_idx=uid)[0] # model.rank: item rank, item_score
        except:
          item_rank = model.rank(user_idx=int(uid))[0]
        # collect top N items
        item_rank_top = item_rank[:n]
        for iid in item_rank_top:
          item_id = list(model.train_set.item_ids)[iid]
          top_n[int(user_id)].append((int(item_id), model.score(uid, iid)))
  return top_n

In [26]:
# random recommendation algorithm
def get_top_n_random(n=10):
    print("Random model is selected:")
    top_n = defaultdict(list)
    test_items = list(rs.test_set.iid_map.keys())
    for uid in rs.train_set.uid_map.values():
      if uid not in top_n.keys():
        user_id = list(rs.train_set.user_ids)[uid]
        for i in range(0, n):
          top_n[int(user_id)].append((int(rd.choice(test_items)), i))
    return top_n

# Gender

In [27]:

mapped_ratings_with_gender = pd.read_csv("https://raw.githubusercontent.com/SavvinaDaniil/BiasInRecommendation/main/data/processed/books/mapped_ratings.csv", index_col=0)
gender_dict = {}
for isbn in mapped_ratings_with_gender.ISBN.unique():
  gender_dict[isbn] = mapped_ratings_with_gender.gender[mapped_ratings_with_gender.ISBN == isbn].iloc[0]

In [28]:
len(gender_dict.keys()) #confirm

6921

In [73]:
for i in range(0, len(algo_names)):
  male_oriented["new_male_female_difference_"+algo_names[i]] = 0.0
  female_oriented["new_male_female_difference_"+algo_names[i]] = 0.0
  diverse["new_male_female_difference_"+algo_names[i]] = 0.0

In [74]:
# male_oriented.at[242118	, "new_male_female_difference_Random"] = 0.3

In [76]:
male_oriented.head()

,num_female,num_male,num_other,num_unknown,num_total,ratio_female,male_female_difference,new_male_female_difference_Random,new_male_female_difference_MostPop,new_male_female_difference_UserKNN,new_male_female_difference_MF,new_male_female_difference_PMF,new_male_female_difference_BPR,new_male_female_difference_NMF,new_male_female_difference_WMF,new_male_female_difference_PF,new_male_female_difference_NeuMF,new_male_female_difference_VAECF
242118,1,5,0,1,7,0.14285714,0.57142857,-0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60569,1,5,0,1,7,0.14285714,0.57142857,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96954,1,5,0,1,7,0.14285714,0.57142857,-0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177040,3,11,0,0,14,0.21428571,0.57142857,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30985,4,16,0,1,21,0.19047619,0.57142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
algo_names = ['Random', 'MostPop', 'UserKNN', 'MF', 'PMF', 'BPR', 'NMF', 'WMF', 'PF', 'NeuMF', 'VAECF']

i = 0

for i in range(0, len(algo_names)):
    male_or_diff = 0
    diverse_diff = 0
    female_or_diff = 0
    
    if algo_names[i] == 'Random':
      top_n = get_top_n_random(n=10)
    elif algo_names[i] == 'UserKNN':
      top_n = get_top_n_UserKNN(n=10)
    else:
      top_n = get_top_n(algo_names[i], n=10)

    users = top_n.keys()
    user_recommendations = {}
    for user in users:
      user_recommendations[user] = [x[0] for x in top_n[user]]
      fem_count = 0
      male_count = 0
      for rec in user_recommendations[user]:
        if gender_dict[rec] == "male":
          male_count += 1
        elif gender_dict[rec] == "female":
          fem_count += 1
      gen_dif = (male_count - fem_count)/len(user_recommendations[user])

      if user in male_oriented.index:
        male_oriented.at[user, "new_male_female_difference_"+algo_names[i]] = gen_dif
      elif user in female_oriented.index:
        female_oriented.at[user, "new_male_female_difference_"+algo_names[i]] = gen_dif
      else:
        diverse.at[user, "new_male_female_difference_"+algo_names[i]] = gen_dif

Random model is selected:
MostPop model is selected:
User-KNN model is selected:
MF model is selected:
PMF model is selected:
BPR model is selected:
NMF model is selected:
WMF model is selected:
PF model is selected:
NeuMF model is selected:
VAECF model is selected:


In [81]:
female_oriented.head()

,num_female,num_male,num_other,num_unknown,num_total,ratio_female,male_female_difference,new_male_female_difference_Random,new_male_female_difference_MostPop,new_male_female_difference_UserKNN,new_male_female_difference_MF,new_male_female_difference_PMF,new_male_female_difference_BPR,new_male_female_difference_NMF,new_male_female_difference_WMF,new_male_female_difference_PF,new_male_female_difference_NeuMF,new_male_female_difference_VAECF
10819,17,7,0,1,25,0.68,-0.4,-0.4,-0.5,-0.2,0.0,0.4,-0.5,0.1,0.1,-0.4,-0.4,-0.2
36638,3,1,0,1,5,0.60,-0.4,0.2,-0.5,-0.3,0.2,0.1,-0.5,0.1,-0.5,-0.2,-0.4,-0.2
36811,3,1,0,1,5,0.60,-0.4,-0.6,-0.5,-0.8,0.1,0.0,-0.5,-0.2,-0.6,1.0,-0.4,0.6
227327,3,1,0,1,5,0.60,-0.4,-0.1,-0.5,-0.3,0.2,-0.7,-0.5,0.5,-0.6,0.2,-0.4,-0.5
125039,26,10,0,4,40,0.65,-0.4,-0.4,-0.5,-0.4,-0.1,-0.3,-0.5,-0.2,0.0,0.5,-0.4,0.4


# Save files

In [78]:
save = True

In [79]:
if save:
  from google.colab import drive
  import pickle as pkl
  drive.mount('/content/drive')

  female_oriented.to_csv("/content/drive/My Drive/new_female_oriented.csv")
  diverse.to_csv("/content/drive/My Drive/new_diverse.csv")
  male_oriented.to_csv("/content/drive/My Drive/new_male_oriented.csv")
  

Mounted at /content/drive


# Popularity

In [ ]:
algo_names = ['Random', 'MostPop', 'UserKNN', 'MF', 'PMF', 'BPR', 'NMF', 'WMF', 'PF', 'NeuMF', 'VAECF']

i = 0
low_rec_gap_list = [] # one entry per algorithmus
medium_rec_gap_list = []
high_rec_gap_list = []

for i in range(0, len(algo_names)):
    print(i)
    df_item_dist[algo_names[i]] = 0
    low_rec_gap = 0
    medium_rec_gap = 0
    high_rec_gap = 0
    
    if algo_names[i] == 'Random':
      top_n = get_top_n_random(n=10)
    elif algo_names[i] == 'UserKNN':
      top_n = get_top_n_UserKNN(n=10)
    else:
      top_n = get_top_n(algo_names[i], n=10)
    low_count = 0
    med_count = 0
    high_count = 0
    for uid, user_ratings in top_n.items():
        iid_list = []
        for (iid, _) in user_ratings:
            df_item_dist.loc[iid, algo_names[i]] += 1
            iid_list.append(iid)
        gap = sum(df_item_dist["count"].loc[iid_list]) / len(iid_list)
        if uid in low.index:
            low_rec_gap += gap
            low_count += 1
        elif uid in med.index:
            medium_rec_gap += gap
            med_count += 1
        elif uid in high.index:
            high_rec_gap += gap
            high_count += 1
    low_rec_gap_list.append(low_rec_gap / low_count)
    medium_rec_gap_list.append(medium_rec_gap / med_count)
    high_rec_gap_list.append(high_rec_gap / high_count)
    i += 1 # next algorithm

0
Random model is selected:
1
MostPop model is selected:


KeyboardInterrupt: ignored

## Save files
To save:
1. df_item_dist
2. low_rec_gap_list etc
3. exp.result & exp.metric
4. training user ids

In [ ]:
save = True

In [ ]:
if save:
  from google.colab import drive
  import pickle as pkl
  drive.mount('/content/drive')

  df_item_dist.to_csv("/content/drive/My Drive/item_pop_dist"+addition_1+"_results_Cornac.csv")
  with open("/content/drive/My Drive/experiment_results_cornac"+addition_4+".pkl","wb") as f:
    pkl.dump(exp.result,f)
  with open("/content/drive/My Drive/experiment_metrics_cornac"+addition_4+".pkl","wb") as f:
    pkl.dump(exp.metrics,f)
  with open("/content/drive/My Drive/low_rec_gap_list_cornac"+addition_4+".pkl","wb") as f:
    pkl.dump(low_rec_gap_list,f)
  with open("/content/drive/My Drive/med_rec_gap_list_cornac"+addition_4+".pkl","wb") as f:
    pkl.dump(medium_rec_gap_list,f)
  with open("/content/drive/My Drive/high_rec_gap_list_cornac"+addition_4+".pkl","wb") as f:
    pkl.dump(high_rec_gap_list,f)
  with open("/content/drive/My Drive/training_user_ids"+addition_4+"_cornac.pkl","wb") as f:
    pkl.dump(list(rs.train_set.user_ids),f)

Mounted at /content/drive
